## gurobi求解2019-f范例

In [1]:
import numpy as np
import pandas as pd
import gurobipy as grb

## 一、读取excel表2的数据

In [2]:

df=pd.read_excel('题目/附件1：数据集1-终稿.xlsx', engine='openpyxl',skiprows=[0])

D:\Anaconda3\envs\mathmodel\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
# 构建所有点坐标，0为起点，shape-1为终点； 第二列为x坐标，第三列为y坐标，第四列为z坐标
coordinates = df.values[:, 1:4]  

In [4]:
coordinates

array([[0.0, 50000.0, 5000.0],
       [33070.825830821, 2789.48076085272, 5163.5246804925],
       [54832.8870194109, 49179.2191080384, 1448.30379093285],
       ...,
       [14870.6016822319, 95939.1733623643, 8248.83618468285],
       [93009.5674461602, 4549.3332597056, 7882.60829650082],
       [100000.0, 59652.3433795158, 5022.00116448164]], dtype=object)

In [5]:
# 构建校正向量，1表示垂直校正点，0表示水平校正点，2表示起点，3表示终点
correction_vector = list(df.values[:, 4])
correction_vector[0] = 2
correction_vector[612] = 3

In [6]:
correction_vector

[2,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,


In [7]:
# 计算欧式距离，用矩阵A标记
A = np.zeros((613, 613))
for i in range(613):
    for j in range(613):
        A[i, j] = np.sqrt(
            (coordinates[i, 0] - coordinates[j, 0]) ** 2 + (coordinates[i, 1] - coordinates[j, 1]) ** 2 + (
                        coordinates[i, 2] - coordinates[j, 2]) ** 2)

In [8]:
len(A)

613

In [9]:
# 第一题的基本常量
shape=len(A)
alpha1 = 25
alpha2 = 15
beta1 = 20
beta2 = 25
delta = 0.001
theta = 30

In [10]:
# 将A转化成tupledict类型
dict_A = {}
for i in range(shape):
    for j in range(shape):
        dict_A[i, j] = A[i, j]

In [11]:
dict_A

{(0, 0): 0.0,
 (0, 1): 57641.47281500988,
 (0, 2): 54953.92366444448,
 (0, 3): 79240.62746162385,
 (0, 4): 38627.495328737336,
 (0, 5): 35755.398430861416,
 (0, 6): 25043.323671058613,
 (0, 7): 51047.28532187423,
 (0, 8): 97618.0893733903,
 (0, 9): 30364.35679560231,
 (0, 10): 59914.0577682999,
 (0, 11): 58967.29509964733,
 (0, 12): 83631.21744805193,
 (0, 13): 34392.16821863199,
 (0, 14): 88594.97589355292,
 (0, 15): 38669.21677606868,
 (0, 16): 93777.85197558186,
 (0, 17): 77070.4522714437,
 (0, 18): 97313.84011083808,
 (0, 19): 21346.13667077411,
 (0, 20): 96316.60924291782,
 (0, 21): 99830.53528146358,
 (0, 22): 58568.13032575283,
 (0, 23): 62942.89830298805,
 (0, 24): 59925.15788430486,
 (0, 25): 45637.38272180045,
 (0, 26): 33387.17454854625,
 (0, 27): 50312.317748396235,
 (0, 28): 34792.28293104262,
 (0, 29): 17821.378276879175,
 (0, 30): 73384.4012085318,
 (0, 31): 52920.28619150555,
 (0, 32): 56485.25473427393,
 (0, 33): 49365.29712988649,
 (0, 34): 76128.59440860369,
 (0, 35)

## 二、剪枝过程

In [12]:

# V为垂直校正点集合，H为水平校正点集合.共shape-2个点；起点1个，终点1个
V = []
H = []
for i in range(shape):
    if correction_vector[i] == 1:
        V.append(i)
    elif correction_vector[i] == 0:
        H.append(i)
    else:
        pass

In [13]:

C = np.ones((shape, shape))
for i in range(1, shape - 1):  # 不包含起点和终点
    for j in V:
        if A[i, j] > min(alpha1, alpha2) / delta:
            dict_A[i, j] = 0
            C[i, j] = 0
    for j in H:
        if A[i, j] > min(beta1, beta2) / delta:
            dict_A[i, j] = 0
            C[i, j] = 0

In [14]:
# 终点
for i in range(shape - 1):
    if dict_A[i, shape - 1] > theta / delta:
        dict_A[i, shape - 1] = 0
        C[i, shape - 1] = 0
for i in range(shape):
    C[i, i] = 0

In [15]:
edge = []  # 边集
for i in range(shape):
    for j in range(shape):
        if dict_A[i, j] != 0:
            edge.append((i, j))
        else:
            pass

In [16]:
# 剪枝后剩余的边
len(edge)

31266

## 三、使用gurobi建立模型

In [17]:
# 建立模型
model=grb.Model()

Academic license - for non-commercial use only - expires 2022-08-07
Using license file C:\Users\GDZ\gurobi.lic


In [18]:
# 添加变量
# 添加新变量x[i,j],i=0 ~shape-1,j=0~shape-1
x = model.addVars(shape, shape, vtype=grb.GRB.BINARY, name='x')
# 添加新变量h[i],v[i],i=0 ~shape-1
h = model.addVars(shape, vtype=grb.GRB.CONTINUOUS, name="h")
v = model.addVars(shape, vtype=grb.GRB.CONTINUOUS, name="v")

In [19]:
# 添加限制条件
# x[i,j]==0 根据C矩阵的信息进行剪枝
for i in range(shape):
    for j in range(shape):
        if C[i, j] == 0:
            model.addConstr(x[i, j] == 0)

In [20]:
# 添加限制条件起始点，中间点，终点的出度入度条件
test1 = [0] * shape  # 出度表达式
test2 = [0] * shape  # 入度表达式
# test1[i]表示i节点的出度，起点
for (i, j) in edge:
    test1[i] = test1[i] + x[i, j]
# test2[i]表示i节点的入度，终点
for (j, i) in edge:
    test2[i] = test2[i] + x[j, i]
for i in range(shape):
    if i == 0:
        #起点 一定是出度为1，入度为0
        model.addConstr(test1[i] == 1)
        model.addConstr(test2[i] == 0)
    elif 0 < i < shape - 1:
        #中间点 一定是出度和入度同时为0,或1         
        model.addConstr(test1[i] == test2[i])
    else:
        model.addConstr(test1[i] == 0)
        model.addConstr(test2[i] == 1)

In [21]:
# 添加限制条件对h，v进行约束
for (i, j) in edge:
    #从i点到j点，水平误差应该满足     
    model.addConstr(correction_vector[i] * h[i] + delta * A[i, j] - h[j] <= 10000 - 10000 * x[i, j])
    #从i点到j点，垂直误差应该满足   
    model.addConstr((1 - correction_vector[i]) * v[i] + delta * A[i, j] - v[j] <= 10000 - 10000 * x[i, j])
# h,v代表飞行棋到达i点是误差校正前的水平偏移量和竖直偏移量
for i in range(shape):
    if i == 0:
        model.addConstr(h[i] == 0)
        model.addConstr(v[i] == 0)
    elif 0 < i < shape - 1:
        if correction_vector[i] == 1:
            model.addConstr(h[i] <= alpha2)
            model.addConstr(v[i] <= alpha1)
        else:
            model.addConstr(h[i] <= beta2)
            model.addConstr(v[i] <= beta1)
    else:
        model.addConstr(h[i] <= theta)
        model.addConstr(v[i] <= theta)

In [23]:
# 添加目标函数
# 设置罚函数系数
dict_modify = {}
for i in range(shape):
    for j in range(shape):
        dict_modify[i, j] = 10000
dict_modify = grb.tupledict(dict_modify)
dict_A=grb.tupledict(dict_A)
model.setObjective(dict_A.prod(x) + 1 * dict_modify.prod(x), grb.GRB.MINIMIZE)
model.update()

In [24]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 408876 rows, 376995 columns and 565814 nonzeros
Model fingerprint: 0x4f45c8bb
Variable types: 1226 continuous, 375769 integer (375769 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+04, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 372927 rows and 352712 columns
Presolve time: 1.69s
Presolved: 35949 rows, 24283 columns, 127155 nonzeros
Variable types: 611 continuous, 23672 integer (23672 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.02s

Solved with dual simplex

Root relaxation: objective 1.646440e+05, 561 iterations, 0.17 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node

In [25]:
for s in model.getVars():
    if s.x == 1:
        print('%s''%g' % (s.varName, s.x))
print('Obj:%g' % model.objVal)

x[0,503]1
x[91,607]1
x[250,340]1
x[277,612]1
x[294,91]1
x[340,277]1
x[503,294]1
x[540,250]1
x[607,540]1
Obj:194861


## 绘制3D图

In [26]:
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

In [27]:
node = [0, 503, 294, 91, 607, 540, 250, 340, 277, 612]

In [29]:
%matplotlib widget
fig = plt.figure(figsize=plt.figaspect(1)*1.7)
ax = Axes3D(fig)
# 起点，终点为黄色，中间点1为绿+，0为蓝^
for i in range(613):
    if i == 0:
        ax.scatter(coordinates[i, 0], coordinates[i, 1], coordinates[i, 2], c='r', marker='^')
    elif 0 < i < 612 and correction_vector[i] == 1:
        ax.scatter(coordinates[i, 0], coordinates[i, 1], coordinates[i, 2], c='g', marker='.')
    elif 0 < i < 612 and correction_vector[i] == 0:
        ax.scatter(coordinates[i, 0], coordinates[i, 1], coordinates[i, 2], c='b', marker='.')
    else:
        ax.scatter(coordinates[i, 0], coordinates[i, 1], coordinates[i, 2], c='r', marker='^')
        
for i in range(len(node) - 1):
    ax.plot([coordinates[node[i], 0], coordinates[node[i + 1], 0]],
            [coordinates[node[i], 1], coordinates[node[i + 1], 1]],
            [coordinates[node[i], 2], coordinates[node[i + 1], 2]], c='k', linewidth=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …